In [7]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# get json tesla data
chargers = pd.read_json('input/tesla-chargers.json')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(f"Columns List: {chargers.columns}")
print(chargers.shape)

Columns List: Index(['address_line_1', 'address_line_2', 'address_notes', 'address',
       'amenities', 'baidu_lat', 'baidu_lng', 'chargers', 'city',
       'common_name', 'country', 'destination_charger_logo',
       'destination_website', 'directions_link', 'emails', 'geocode', 'hours',
       'is_gallery', 'kiosk_pin_x', 'kiosk_pin_y', 'kiosk_zoom_pin_x',
       'kiosk_zoom_pin_y', 'latitude', 'location_id', 'location_type',
       'longitude', 'nid', 'open_soon', 'path', 'postal_code',
       'province_state', 'region', 'sales_phone', 'sales_representative',
       'sub_region', 'title', 'trt_id'],
      dtype='object')
(15737, 37)


In [2]:
# just USA chargers
#q = "country == 'United States' & province_state == 'KY' & location_type.str.contains('supercharger', regex=False)"
q = "country == 'United States' & location_type.str.contains('supercharger', regex = False)"

us_chargers = chargers.query(q)

us_chargers = us_chargers[['title', 'location_id', 'nid', 
                           'address_line_1', 'city', 'province_state', 'postal_code',
                           'chargers', 'common_name', 'open_soon',
                           'latitude', 'longitude', 'location_type']]

# rename
us_chargers.rename(columns = {"province_state": "state_code", "postal_code": "zip_code", "address_line_1" : "address"}, inplace = True)

print(us_chargers.shape)
us_chargers.head()

(1351, 13)


,title,location_id,nid,address,city,state_code,zip_code,chargers,common_name,open_soon,latitude,longitude,location_type
2,"Albany, NY Supercharger",albanysupercharger,6293,131 Colonie Center,Albany,NY,12205,<p><strong>Charging</strong><br />6 Supercharg...,Colonie Center,0,42.710356,-73.819109,"[supercharger, standard charger]"
3,"Albert Lea, MN Supercharger",albertleasupercharger,6294,811 E Plaza St,Albert Lea,MN,56007,<p><strong>Charging</strong><br />4 Supercharg...,AmericInn Lodge & Suites Albert Lea,0,43.686060,-93.357721,"[supercharger, standard charger]"
4,"Angola, IN Supercharger",angolasupercharger,6298,3855 Indiana 127,Angola,IN,46703,<p><strong>Charging</strong><br />6 Supercharg...,Ramada Angola,0,41.699048,-85.000326,"[supercharger, standard charger]"
6,"Atascadero, CA Supercharger",atascaderosupercharger,6300,6950 El Camino Real,Atascadero,CA,93422,<p><strong>Charging</strong><br />8 Supercharg...,Rabobank,0,35.486585,-120.666378,"[supercharger, standard charger]"
11,"Beaver, UT Supercharger",beaversupercharger,6308,Littlefield Interchange,Beaver,UT,84713,<p><strong>Charging</strong><br/>8 Supercharge...,Beaver Valley Chevron,0,38.249149,-112.652524,"[supercharger, standard charger]"


In [3]:
# get the location_id, open date from the supercharge.info

open_info = pd.read_json('input/supercharge-info.json')
open_info = open_info[['id', 'locationId', 'dateOpened']]

# rename
open_info.rename(columns = {"locationId": "location_id", "dateOpened": "date_opened"}, inplace = True)

print(open_info.shape)
open_info.head()

(2550, 3)


,id,location_id,date_opened
0,2215,dangjinsupercharger,2019-12-20
1,2216,seoulgangseosupercharger,2019-12-20
2,122,sanjuancapistranosupercharger,2014-05-06
3,125,glenwoodspringssupercharger,2013-09-17
4,126,grandjunctionsupercharger,2014-01-10


In [4]:
# merge the two dataset to add open date to the data
us_chargers = pd.merge(us_chargers, open_info, how = 'left', on = 'location_id')

display(us_chargers.shape)
display(us_chargers.head())
display(us_chargers.dtypes)

(1352, 15)

,title,location_id,nid,address,city,state_code,zip_code,chargers,common_name,open_soon,latitude,longitude,location_type,id,date_opened
0,"Albany, NY Supercharger",albanysupercharger,6293,131 Colonie Center,Albany,NY,12205,<p><strong>Charging</strong><br />6 Supercharg...,Colonie Center,0,42.710356,-73.819109,"[supercharger, standard charger]",160.0,2014-04-30
1,"Albert Lea, MN Supercharger",albertleasupercharger,6294,811 E Plaza St,Albert Lea,MN,56007,<p><strong>Charging</strong><br />4 Supercharg...,AmericInn Lodge & Suites Albert Lea,0,43.686060,-93.357721,"[supercharger, standard charger]",152.0,2014-01-15
2,"Angola, IN Supercharger",angolasupercharger,6298,3855 Indiana 127,Angola,IN,46703,<p><strong>Charging</strong><br />6 Supercharg...,Ramada Angola,0,41.699048,-85.000326,"[supercharger, standard charger]",147.0,2013-12-09
3,"Atascadero, CA Supercharger",atascaderosupercharger,6300,6950 El Camino Real,Atascadero,CA,93422,<p><strong>Charging</strong><br />8 Supercharg...,Rabobank,0,35.486585,-120.666378,"[supercharger, standard charger]",108.0,2013-07-03
4,"Beaver, UT Supercharger",beaversupercharger,6308,Littlefield Interchange,Beaver,UT,84713,<p><strong>Charging</strong><br/>8 Supercharge...,Beaver Valley Chevron,0,38.249149,-112.652524,"[supercharger, standard charger]",187.0,2014-04-14


title             object
location_id       object
nid                int64
address           object
city              object
state_code        object
zip_code          object
chargers          object
common_name       object
open_soon          int64
latitude         float64
longitude        float64
location_type     object
id               float64
date_opened       object
dtype: object

In [5]:
# Add State Fullname 

# read the csv into a dict for use in replace map
state_codes = pd.read_csv('input/us_state_codes.csv')
state_codes.head()

us_chargers = pd.merge(us_chargers, state_codes, how = 'inner', on = 'state_code')
us_chargers.head()

#us_chargers_df.state.value_counts()

,title,location_id,nid,address,city,state_code,zip_code,chargers,common_name,open_soon,latitude,longitude,location_type,id,date_opened,state_name
0,"Albany, NY Supercharger",albanysupercharger,6293,131 Colonie Center,Albany,NY,12205,<p><strong>Charging</strong><br />6 Supercharg...,Colonie Center,0,42.710356,-73.819109,"[supercharger, standard charger]",160.0,2014-04-30,New York
1,"JFK - Queens, NY Supercharger",jfksupercharger,6407,150th Ave and 147th St,Queens,NY,11430,<p><strong>Charging</strong><br />4 Supercharg...,JFK Airport Travel Plaza,0,40.663190,-73.793192,"[supercharger, standard charger]",161.0,2013-12-23,New York
2,Utica Supercharger,uticanysupercharger,7654,50 Auert Ave.,Utica,NY,13501,<p><strong>Charging</strong><br />8 supercharg...,North Utica Shopping Center,0,43.113878,-75.206857,"[supercharger, standard charger]",545.0,2015-02-11,New York
3,Syracuse Supercharger,syracusesupercharger,9018,441 Electronics Parkway,Liverpool,NY,13088,<p><strong>Charging</strong><br />8 supercharg...,Holiday Inn Syracuse/Liverpool,0,43.102424,-76.187446,"[supercharger, standard charger]",547.0,2015-04-16,New York
4,Buffalo Supercharger,buffalosupercharger,9563,4455 Transit Road,Buffalo,NY,14221,<p><strong>Charging</strong><br />8 supercharg...,Shops at Main / Transit,0,42.968675,-78.695680,"[supercharger, standard charger]",415.0,2015-06-11,New York


# Bar Charts


In [20]:
# Number of Chargers by State

# Only get open chargers, I need to check this. It says there 
# are 100 chargers opening in CA this year, that is a lot
q = 'open_soon == 0'
open_us_chargers = us_chargers.query(q)

# get the top ten
top_ten = open_us_chargers['state_name'].value_counts()[:10]
top_ten

California      199
Florida          54
New York         48
Texas            47
New Jersey       31
Washington       29
Virginia         29
Pennsylvania     26
Colorado         25
Illinois         25
Name: state_name, dtype: int64

In [68]:
%matplotlib widget

xAxis = top_ten.index.values.tolist() 
yAxis = top_ten.values.tolist()

# tesla red
colors = ['#e82127']

# make a bar chart
plt.bar(xAxis, yAxis, color = colors, label = 'Chargers', zorder = 10)

# text
plt.title('Top Ten Supercharger States', fontsize = 18)

plt.xlabel('State', fontsize = 8, fontweight = 'bold')
plt.xticks(rotation = 45, fontsize = 8)

plt.ylabel('Number of Supercharger Stations', fontsize = 8, fontweight = 'bold')
plt.yticks(fontsize = 8)

plt.legend()

# totally ripped off, i need to better understand what is happening 
for i in range(len(yAxis)):
    plt.annotate(str(yAxis[i]), xy=(xAxis[i],yAxis[i]), ha='center', va='bottom')
    
# adjust the subplot so the text doesn't run off the image
plt.subplots_adjust(bottom = 0.25)

## TODO: add space above the 199 bar for CA

# grids on
plt.grid(color = 'gray', linestyle = 'solid', alpha = 0.1, zorder = 0.1)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …